In [1]:
import os
import random
import itertools
import json
from pathlib import Path
import sys
sys.path.append("../src")

import cv2
import numpy as np
import torch
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.config.config import get_cfg
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.evaluation.coco_evaluation import instances_to_coco_json
from ensemble_boxes import soft_nms


from defs import ROOT, TRAIN_DATA_PATH, TRAIN_IMAGES_PATH, TRAIN_MASKS_PATH, VAL_DATA_PATH, VAL_IMAGES_PATH, VAL_MASKS_PATH
from dt2.register import register_my_dataset
from dt2.visualize import visualize_dataset_dict, visualize_batch_item
from inference.predictor import get_predictor
from inference.submit import prepare_submit, get_predictions_for_submit, prepare_detection_submit, prepare_segmentation_submit
from inference.grid_search import run_grid_search
from inference.ensemble import convert_instances_to_wbf_format, convert_wbf_format_to_instances
from train import Trainer
from baseline.evaluation.evaluation import calculation_map_050, calculate_meanIOU, competition_metric


In [2]:
exp_names = ["FPN-res101-bs=6-multiscale", "panopticFPN-resnet50-bs=6-multiscale", "FPN-cascade-res50-GN-bs=2-multiscale"]
exp_dirs = [ROOT / "outputs" / exp_name for exp_name in exp_names]

In [3]:
score_threshes = [0.0001] * len(exp_dirs)
nms_threshes = [0.5] * len(exp_dirs)
ensemble_method = "wbf"
ensemble_method_kwargs = {"iou_thr": 0.6, "weights": [1, 1, 1]}

prepare_submit(VAL_IMAGES_PATH, Path("."), exp_dirs, score_threshes, nms_threshes, ensemble_method, **ensemble_method_kwargs)
competition_metric(str(VAL_DATA_PATH / "coco.json"), "detection_predictions.json",
                   str(VAL_DATA_PATH / "segmentation_gt.json"), "segmentation_predictions.json")

PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
segmentation submit preparation: 100%|██████████| 411/411 [00:00<00:00, 208548.14it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=8.73s).
Accumulating evaluation results...
DONE (t=1.52s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.299
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.630
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.245
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.235
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.357
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.233
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.459
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

100%|██████████| 411/411 [00:24<00:00, 16.71it/s]

Detection: 0.6299162038346096
Segmentation: 0.8877118715327906


0.70725

## Grid search

In [2]:
exp_names = ["FPN-res101-bs=6-multiscale", "panopticFPN-resnet50-bs=6-multiscale", "FPN-cascade-res50-GN-bs=2-multiscale"]
exp_dirs = [ROOT / "outputs" / exp_name for exp_name in exp_names]

In [3]:
score_threshes = [[0.0001] * len(exp_dirs)]
nms_threshes = [[0.5] * len(exp_dirs)]
ensemble_methods = ["wbf"]
ensemble_methods_kwargs = [{"iou_thr": 0.6, "weights": [1, 1, 1]}, {"iou_thr": 0.6, "weights": [1.5, 1, 1]}]
metrics = run_grid_search(VAL_IMAGES_PATH, Path("."), exp_dirs, score_threshes, nms_threshes, ensemble_methods, ensemble_methods_kwargs)

PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
100%|██████████| 411/411 [09:26<00:00,  1.38s/it]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=8.37s).
Accumulating evaluation results...
DONE (t=1.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.299
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.630
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.245
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.235
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.357
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.233
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.459
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
100%|██████████| 411/411 [09:16<00:00,  1.35s/it]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.98s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=8.55s).
Accumulating evaluation results...
DONE (t=1.49s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.630
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.236
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.345
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.362
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.234
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.460
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets